- find new data
- load the model
- calc feats & infer on data. need to cut data up
- also try calc feats then infer in batches
- then make it so it outputs the predictions, so I can timestamp it

- write as notebook but remember it will run on hpc!

In [ ]:
# Downgrade tensorflow - run this cell, restart the runtime then no need to run this cell again
!pip install tensorflow==2.8.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.4/498.4 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 KB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: keras
    Found existing installation: keras 2.11.0
    Uninstalling keras-2.11.0:
      Successfully uninstalled keras-2.11.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.11.2
    Uninstalling tensorboard-2.11.2:
      Successf

In [ ]:
# check the downgraded version of tf is installed, should print 2.8.3
import tensorflow as tf
print(tf.__version__)

2.8.3


In [ ]:
# Install autokeras - still required for loading the model
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 1.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.1/578.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Connect GDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ML imports
import tensorflow as tf
from tensorflow.keras.models import load_model
import autokeras

# import general packages
import librosa # for audio procesing
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import datetime

In [ ]:
# set paths to where audio is saved
new_audio_dir = "/content/drive/MyDrive/bombs/data/final_dataset/deployment_data/" #make sure ends in '/'
model_dir = "/content/drive/MyDrive/bombs/data/final_dataset/autokeras_best_model/best_model"

# enter sample rate of your audio files - these will get resampled to 8000 if needed
sample_rate = 16000

# load the tensorflow model
best_model = load_model(model_dir)

# find the audio
new_audio_files = [f for f in os.listdir(new_audio_dir) if f.endswith('.wav') or f.endswith('.WAV')]
print("Number of files found: " + str(len(new_audio_files)))

Number of files found: 2


In [ ]:
def model_inference(dataset, dir, sample_rate):
  
  #set the sample rate that should be used (this will trigger resampling of audio if needed)
  new_sample_rate = 8000

  # set the length of each window in samples
  window_length_samples = int(2.88 * new_sample_rate)
  stutter = int(window_length_samples/2)

  # create an emppty list where suspected bombs will be stored
  suspected_bombs = [] ############################################################### needed?

  # first loop 
  for f in tqdm(dataset):
      # Load audio
      audio_path = os.path.join(dir, f)
      audio, sample_rate = librosa.load(path=audio_path, sr=sample_rate)

      # resample the audio if needed
      if sample_rate != new_sample_rate:
        audio = librosa.resample(audio, sample_rate, new_sample_rate)

      # Compute the number of windows
      num_windows = len(audio) // window_length_samples

      # Iterate over windows and compute the MFCC spectrogram for each window
      for i in range(num_windows):

          # Get the start and end indices for the current window
          start_index1 = i * window_length_samples
          start_index2 = (i * window_length_samples) + stutter

          end_index1 = (i + 1) * window_length_samples
          end_index2 = ((i + 1) * window_length_samples) + stutter

          # Extract the current window
          window1 = audio[start_index1:end_index1]
          window2 = audio[start_index2:end_index2]

          # Compute the MFCC spectrogram for the current window
          mfcc_spec1 = librosa.feature.mfcc(y=window1, sr=new_sample_rate, n_mfcc = 32)
          mfcc_spec2 = librosa.feature.mfcc(y=window2, sr=new_sample_rate, n_mfcc = 32)

          # add extra dimensions for networks input_shape
          mfcc_spec1 = np.expand_dims(mfcc_spec1, axis=2)
          mfcc_spec2 = np.expand_dims(mfcc_spec2, axis=2)
          mfcc_spec1 = np.expand_dims(mfcc_spec1, axis=0)
          mfcc_spec2 = np.expand_dims(mfcc_spec2, axis=0)

          # Perform the function on the MFCC spectrogram
          result1 = best_model.predict(mfcc_spec1) > 0.5
          result2 = best_model.predict(mfcc_spec2) > 0.5

          if result1 == True or result2 == True:          
            timestamp_seconds = i * 2.88
            timestamp_str = str(datetime.timedelta(seconds=timestamp_seconds))
            print()
            print(f)
            print(timestamp_str)
            # adapt here so that it prints 5sec either side (or something of this)
            # why is it printing the timestamp for both result1 and result2 around bombs?
            # ask chatgpt how to return the part of the file I want
            #ask chatgpt how this code could run faster
  return None

huh = model_inference(new_audio_files, new_audio_dir, sample_rate)

  0%|          | 0/2 [00:00<?, ?it/s]


20220915_151000.WAV
0:00:23.040000

20220915_151000.WAV
0:00:40.320000

20220915_151000.WAV
0:00:43.200000


 50%|█████     | 1/2 [00:07<00:07,  7.35s/it]


20220830_150600.WAV
0:00:20.160000

20220830_150600.WAV
0:00:23.040000

20220830_150600.WAV
0:00:48.960000


100%|██████████| 2/2 [00:11<00:00,  5.63s/it]
